## Processing of historical sales 
Process ZIP files of [Property Sales][sales] downloaded to `data/raw/not_synced`. 
* Files are saved in a directory mess 
* File names are prefixed with a three digit code for the LGA. Only consider the following:
    * `090` - Ryde
    * `139` - Canada Bay
    * `260` - City of Parramatta
* The [Sales Data Format][sales_format_pdf] describes the format of the DAT files
* There is a change in the ZIP files 
    * 2001 to 2014 - each ZIP folder has sub-folders;
    * 2015 onwards - each ZIP folder has a ZIP file for each week...



[sales]: https://valuation.property.nsw.gov.au/embed/propertySalesInformation
[sales_format_pdf]: https://www.valuergeneral.nsw.gov.au/__data/assets/pdf_file/0015/216402/Current_Property_Sales_Data_File_Format_2001_to_Current.pdf

In [20]:
import zipfile
from pathlib import Path
import io
from typing import Dict, Tuple, Optional
import pandas as pd
TESTING = False

def process_dat(file_name: str, file_bytes: bytes, df_properties_all: pd.DataFrame, df_sales_all: pd.DataFrame) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Parse NSW Property Sales DAT file bytes into normalized relational tables:
    - properties (PK: property_id)
    - sales      (PK: sale_counter)

    Args:
    file_name (str)     : the name of the DAT file in the ZIP archive 
    file_bytes (bytes)  : the byte data in actual file
    df_properties_all   : Data frame of all properties
    df_sales_all        : Data frame of all sales
    Returns: (df_properties_all, df_sales_all) - with the new information
    """

    text = file_bytes.decode("ascii", errors="replace")
    lines = text.splitlines()

    # properties keyed by property_id 
    properties_by_id: Dict[str, Dict] = {}

    # sales keyed by sale_counter 
    sales_by_counter: Dict[str, Dict] = {}

    # track the "current" sale to attach subsequent C records
    current_sale_counter: Optional[str] = None
    
    # if there is no unit number don't accept the current sale
    accept_current_sale = False

    def _none_if_empty(s: str) -> Optional[str]:
        return s if s != "" else None

    def _update_if_value(target: Dict, key: str, value: Optional[str]) -> None:
        # Keep existing non-null unless new value is non-null; update on new non-null
        if value is not None:
            target[key] = value

    for line in lines:
        # Remove trailing delimiter and split
        fields = line.rstrip(";").split(";")
        if not fields:
            continue

        rec = fields[0]

        # A = header, Z = trailer; ignore for table outputs
        if rec == "A":
            continue
        if rec == "Z":
            break

        # D exists but contains suppressed owner details; ignore
        if rec == "D":
            continue

        # --------------------------
        # B = property + sale record
        # --------------------------
        if rec == "B":
            # Field order per spec 
            #  0 rec_type
            #  1 district_code
            #  2 property_id
            #  3 sale_counter
            #  4 download_datetime
            #  5 property_name
            #  6 unit_number
            #  7 house_number
            #  8 street_name
            #  9 locality
            # 10 postcode
            # 11 area
            # 12 area_type
            # 13 contract_date (YYYYMMDD)
            # 14 settlement_date (YYYYMMDD)
            # 15 purchase_price
            # 16 zoning
            # 17 nature_of_property
            # 18 primary_purpose
            # 19 strata_lot_number
            # 20 component_code
            # 21 sale_code
            # 22 interest_percent
            # 23 dealing_number

            district_code = fields[1]
            property_id = _none_if_empty(fields[2].strip())
            sale_counter = fields[3]
            
            # code to reject sales that 
            unit_number = _none_if_empty(fields[6].strip())     # Unit 
            accept_current_sale = unit_number is not None and property_id is not None
            if not accept_current_sale:
                current_sale_counter = None
                continue

            current_sale_counter = sale_counter

            # --- upsert properties row (PK: property_id) ---
            prop = properties_by_id.get(property_id)
            if prop is None:
                prop = {
                    "district_code": district_code,
                    "property_id": property_id,  # include for convenience, though PK is property_id
                    "property_name": None,
                    "unit_number": None,
                    "house_number": None,
                    "street_name": None,
                    "locality": None,
                    "postcode": None,
                    "area": None,
                    "area_type": None,
                    "zoning": None,
                    "nature_of_property": None,
                    "primary_purpose": None,
                    "strata_lot_number": None,
                    "legal_description": "",  # concatenated from C records
                }
                properties_by_id[property_id] = prop
            else:
                # If district_code differs across files, keep the first; you can change this rule if needed.
                prop.setdefault("district_code", district_code)

            _update_if_value(prop, "property_name", _none_if_empty(fields[5]))
            _update_if_value(prop, "unit_number", _none_if_empty(fields[6]))
            _update_if_value(prop, "house_number", _none_if_empty(fields[7]))
            _update_if_value(prop, "street_name", _none_if_empty(fields[8]))
            _update_if_value(prop, "locality", _none_if_empty(fields[9]))
            _update_if_value(prop, "postcode", _none_if_empty(fields[10]))
            _update_if_value(prop, "area", _none_if_empty(fields[11]))
            _update_if_value(prop, "area_type", _none_if_empty(fields[12]))
            _update_if_value(prop, "zoning", _none_if_empty(fields[16]))
            _update_if_value(prop, "nature_of_property", _none_if_empty(fields[17]))
            _update_if_value(prop, "primary_purpose", _none_if_empty(fields[18]))
            _update_if_value(prop, "strata_lot_number", _none_if_empty(fields[19]))

            # --- create sales row (PK: sale_counter) ---
            # If duplicates exist, we keep the first and continue appending legal desc.
            sales = sales_by_counter.get(sale_counter)
            if sales is None:
                sales = {
                    "sale_counter": sale_counter,
                    "property_id": property_id,
                    "contract_date": _none_if_empty(fields[13]),
                    "settlement_date": _none_if_empty(fields[14]),
                    "purchase_price": _none_if_empty(fields[15]),
                    "component_code": _none_if_empty(fields[20]),
                    "sale_code": _none_if_empty(fields[21]),
                    "interest_percent": _none_if_empty(fields[22]),
                    "dealing_number": _none_if_empty(fields[23]),
                }
                sales_by_counter[sale_counter] = sales

            continue

        # ---------------------------------
        # C = legal description (may repeat)
        # ---------------------------------
        if rec == "C":
            if not accept_current_sale:
                continue
            # Field order per spec 
            # 0 rec_type, 1 district_code, 2 property_id, 3 sale_counter, 4 download_datetime, 5 legal_desc_part
            if len(fields) >= 6:
                property_id = fields[2]
                legal_part = fields[5] or ""
                properties_by_id[property_id]["legal_description"] = properties_by_id[property_id]["legal_description"] + legal_part
            continue

    # -----------------------
    # Build final DataFrames
    # -----------------------
    properties_df = pd.DataFrame(properties_by_id.values())

    sales_df = pd.DataFrame(sales_by_counter.values())

    # Type conversions
    # Dates are YYYYMMDD per spec 
    if not sales_df.empty:
        sales_df["contract_date"] = pd.to_datetime(sales_df["contract_date"], format="%Y%m%d", errors="coerce")
        sales_df["settlement_date"] = pd.to_datetime(sales_df["settlement_date"], format="%Y%m%d", errors="coerce")
        sales_df["purchase_price"] = pd.to_numeric(sales_df["purchase_price"], errors="coerce")

    if not properties_df.empty:
        properties_df["area"] = pd.to_numeric(properties_df["area"], errors="coerce")

    # Optional: enforce uniqueness expectations
    # (won't raise; just ensures the PK columns are unique in the outputs)
    if not properties_df.empty:
        properties_df = properties_df.drop_duplicates(subset=["property_id"], keep="first").reset_index(drop=True)

    if not sales_df.empty:
        sales_df = sales_df.drop_duplicates(subset=["sale_counter"], keep="first").reset_index(drop=True)

    # Now add the data to the master dataframes
    # Append and de-duplicate PROPERTIES by primary key: property_id
    key = "property_id"
    if df_properties_all.empty:
        df_properties_all = properties_df.copy()
    else:
        all_cols = sorted(set(df_properties_all.columns) | set(properties_df.columns))
        df_properties_all = df_properties_all.reindex(columns=all_cols)
        df_props = properties_df.reindex(columns=all_cols)

        merged = df_properties_all.merge(df_props, on=key, how="outer", suffixes=("", "__new"))

        for col in all_cols:
            if col == key:
                continue
            new_col = f"{col}__new"
            if new_col in merged.columns:
                merged[col] = merged[new_col].where(merged[new_col].notna(), merged[col])
                merged.drop(columns=[new_col], inplace=True)

        df_properties_all = merged


    # Append and de-duplicate SALES by composite key: (property_id, sale_counter, settlement_date)
    if df_sales_all.empty:
        df_sales_all = sales_df.copy()
    else:
        df_sales_all = pd.concat([df_sales_all, sales_df], ignore_index=True)

        df_sales_all = (
            df_sales_all
            .drop_duplicates(subset=["property_id", "sale_counter", "settlement_date"], keep="last")
            .reset_index(drop=True)
        )
    return df_properties_all, df_sales_all

def process_zip_folder(zip_folder: str, prefixes: tuple=("090","139","260")) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    process_zip_folder -> processes the supplied folder of ZIP files 
    Each file may contain either more ZIP files or DAT files in a folder structure

    Args:
        zip_folder (str): the folder containing the ZIP files
        prefixes (str, optional): The prefixes of DAT files to include. Defaults to ["090","139","260"].

    Return:
        properties_all_df
        sales_all_df
    """
    # Set the path to be the folder passed in
    zip_folder_path = Path(zip_folder)
    # convert prefixes to a tuple
    prefix_tuple = tuple(prefixes)
    # --- initialise properties and sales data frames ---
    df_properties_all = pd.DataFrame()
    df_sales_all = pd.DataFrame()

    # loop through the ZIP files in the folder (not subfolders) - to make it 
    #   recurse through subfolders use rglob not glob
    for zip_path in zip_folder_path.glob("*.zip"):
        print(f"Opening ZIP: {zip_path.name}")
        if TESTING and zip_path.name != "2024.zip":
            continue

        with zipfile.ZipFile(zip_path, "r") as zf:
            for zip_info in zf.infolist():
                # Skip directories
                if zip_info.is_dir():
                    continue 
                    
                file_name = Path(zip_info.filename).name
                if TESTING and file_name != "20240101.zip":
                    continue

                # Check is the FILE a DAT file and starts with the right type
                if file_name.lower().endswith(".dat") and file_name.startswith(prefixes):
                    with zf.open(zip_info) as dat_file:
                        file_bytes = dat_file.read()
                        process_dat(file_name, file_bytes, df_properties_all,df_sales_all)

                # Check if i the file is ZIP and if so scan in it...
                if file_name.lower().endswith(".zip"):
                    # recursive call the function...
                    print(f"Opening contained ZIP: {file_name}")
                    inner_bytes = zf.read(zip_info)
                    with zipfile.ZipFile(io.BytesIO(inner_bytes),"r") as inner_zf:
                        for inner_info in inner_zf.infolist():
                            if inner_info.is_dir():
                                continue
                            inner_name = Path(inner_info.filename).name
                            if inner_name.lower().endswith(".dat") and inner_name.startswith(prefixes):
                                with zf.open(zip_info) as dat_file:
                                    file_bytes = inner_zf.read(inner_info)
                                    df_properties_all, df_sales_all = process_dat(inner_name, file_bytes, df_properties_all, df_sales_all)
                                    # print("<<Sales>>:",df_sales.head())
                                    # print("<<Properties>>",df_props.head()

                                    
    return df_properties_all, df_sales_all
                                
if __name__ == "__main__":
    prefixes = ("090","139","260")
    df_properties_all, df_sales_all = process_zip_folder("../data/raw/not_synced",prefixes)
    out_dir = Path("../data/raw")
    zip_path = out_dir / "historical_sales_to_2025.zip"
    with zipfile.ZipFile(zip_path, mode="w", compression=zipfile.ZIP_DEFLATED) as zf:
        zf.writestr("properties.csv", df_properties_all.to_csv(index=False))
        zf.writestr("sales.csv", df_sales_all.to_csv(index=False))
    print(f"Wrote {zip_path}")

Opening ZIP: 2024.zip
Opening contained ZIP: 20240101.zip
Opening contained ZIP: 20240108.zip
Opening contained ZIP: 20240115.zip
Opening contained ZIP: 20240122.zip
Opening contained ZIP: 20240129.zip
Opening contained ZIP: 20240205.zip
Opening contained ZIP: 20240212.zip
Opening contained ZIP: 20240219.zip
Opening contained ZIP: 20240226.zip
Opening contained ZIP: 20240304.zip
Opening contained ZIP: 20240311.zip
Opening contained ZIP: 20240318.zip
Opening contained ZIP: 20240325.zip
Opening contained ZIP: 20240401.zip
Opening contained ZIP: 20240408.zip
Opening contained ZIP: 20240415.zip
Opening contained ZIP: 20240422.zip
Opening contained ZIP: 20240429.zip
Opening contained ZIP: 20240506.zip
Opening contained ZIP: 20240513.zip
Opening contained ZIP: 20240520.zip
Opening contained ZIP: 20240527.zip
Opening contained ZIP: 20240603.zip
Opening contained ZIP: 20240610.zip
Opening contained ZIP: 20240617.zip
Opening contained ZIP: 20240624.zip
Opening contained ZIP: 20240701.zip
Openin

In [15]:
df_properties_all


,property_id,area,area_type,district_code,house_number,legal_description,locality,nature_of_property,postcode,primary_purpose,property_name,strata_lot_number,street_name,unit_number,zoning
0,,NaN,None,260,180,65/SP10662369/SP10662370/SP10662372/SP10662374...,PARRAMATTA,R,2150,RESIDENCE,None,None,GEORGE ST,6335,None
1,1310076,NaN,None,260,17,20/SP59118,NEWINGTON,R,2127,RESIDENCE,None,20,BLAXLAND AVE,20,None
2,1313778,338.0,M,260,50,5/SP51945,SILVERWATER,R,2128,RESIDENCE,None,5,DERBY ST,5,None
3,1901803,NaN,NaN,139,55,6/SP7700,DRUMMOYNE,R,2047,RESIDENCE,NaN,6,GIPPS ST,6,NaN
4,1911815,NaN,NaN,139,18,1/SP7180,DRUMMOYNE,R,2047,RESIDENCE,NaN,1,WOLSELEY ST,1,NaN
5,2843647,NaN,NaN,139,4,239/SP66339,CHISWICK,R,2046,RESIDENCE,RIVIERA APARTMENTS,239,BECHERT RD,239,NaN
6,2856989,NaN,None,260,14,11/SP63224,GRANVILLE,R,2142,RESIDENCE,None,11,HIGH ST,11,None
7,3004220,NaN,None,260,34,2/SP69954,PARRAMATTA,R,2150,RESIDENCE,None,2,FENNELL ST,2,None
8,3054208,NaN,None,260,13,24/SP71875,CARLINGFORD,R,2118,RESIDENCE,None,24,THALLON ST,24,None
9,3111723,NaN,None,260,18,64/SP73705,SILVERWATER,R,2128,RESIDENCE,None,64,DAY ST N,64,None


In [16]:
df_sales_all

,sale_counter,property_id,contract_date,settlement_date,purchase_price,component_code,sale_code,interest_percent,dealing_number
0,1,3461773,2023-11-28,2023-12-22,756500,None,None,0,AT729989
1,2,4173612,2023-11-11,2023-12-22,700000,None,None,0,AT731122
2,4,870023,2023-11-14,2023-12-22,1250000,None,None,0,AT733492
3,8,4092793,2023-11-13,2023-12-22,720000,None,None,0,AT731421
4,9,4092793,2023-11-20,2023-12-22,565000,None,None,0,AT731295
...,...,...,...,...,...,...,...,...,...
86,68,3720923,2023-11-21,2023-12-22,650000,None,None,0,AT731218
87,69,3848561,2023-11-09,2023-12-22,450000,None,None,0,AT731116
88,70,3976048,2023-11-10,2023-12-22,775000,None,None,0,AT733269
89,71,3976048,2023-12-20,2023-12-22,740000,None,None,0,AT733599
